# Neural Network

Gebaseerd op 'Stability of Neural Networks - Plotting Codes - Optimised'

In [1]:
'''
Required Packages
'''

import numpy as np
import pandas as pd
import math
import scipy as sc
from scipy.stats import norm
from scipy.special import roots_hermite
import keras
from keras.models import Sequential
from keras.layers import Dense
import keras.optimizers as opt
from keras.constraints import Constraint
import sobol_seq
import time
import tensorflow as tf
from keras import backend as keras_backend
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
from itertools import chain, product 
from tensorflow import keras

In [6]:
num_cores = 32
#No.of.derivative contracts used for static hedging
no_of_options=15

#No of paths
no_of_paths = 500
no_of_paths_test = 500

# Total number of runs neural network static hedge created
no_of_runs = 4

# num_CPU = 1
# num_GPU = 0

# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores,
#                         allow_soft_placement=True, device_count={'CPU': num_CPU, 'GPU': num_GPU})
# session = tf.Session(config=config)
# keras_backend.set_session(session)

##################################################################################
# Stability Analysis of NN Static Hedging and comparison with Carr's Static Hedge#
##################################################################################

#Current Stock Price
S0=1

#Risk Free Interest Rate
r=0.06 

#Continuos Divident rate
delta=0

#Volatility
sigma=0.2

# Time to maturity for the shorter term options = 0.25 yrs 
t=1

# Time to maturity for the target options = 1 yr
T=4

# Size of each step = 0.01
dt=0.01

# The call was struck at the money on 11/08/2020
K=1.1


# No. of sim time points
sim_grid_points = int(t/dt)

#No. of. time points
sim_points = [dt * i for i in range(1, sim_grid_points + 1)]

# Genereer Test / Training set

Taken from 'Stability of Neural Network - Optimised Code'

In [7]:
# Simulated stock prices for Training
df_sim_stocks=pd.DataFrame(np.zeros(no_of_paths * (sim_grid_points + 1)).reshape(sim_grid_points + 1, no_of_paths))

for path in np.arange(0,no_of_paths):
    np.random.seed(path)
    for i in np.arange(1,sim_grid_points + 1):
            df_sim_stocks.loc[0,path]=S0
            df_sim_stocks.loc[i,path]=df_sim_stocks.loc[i-1,path] + (r-delta)*df_sim_stocks.loc[i-1,path]*dt + sigma*df_sim_stocks.loc[i-1,path]*np.sqrt(dt)*np.random.randn()

# Simulated Stocks for Testing            
df_sim_stocks_test=pd.DataFrame(np.zeros(no_of_paths_test * (sim_grid_points + 1)).reshape(sim_grid_points + 1, no_of_paths_test))

for path in np.arange(0,no_of_paths_test):
    np.random.seed(path+no_of_paths)
    for i in np.arange(1,sim_grid_points + 1):
            df_sim_stocks_test.loc[0,path]=S0
            df_sim_stocks_test.loc[i,path]=df_sim_stocks_test.loc[i-1,path] + (r-delta)*df_sim_stocks_test.loc[i-1,path]*dt + sigma*df_sim_stocks_test.loc[i-1,path]*np.sqrt(dt)*np.random.randn()

# Load the simulated stock dataframes to csv
df_sim_stocks.to_csv('Stability Analysis - Outputs/Dataframes Optimized/Stock Simulations/df_sim_stocks_sim' + str(no_of_paths) + '.csv')
df_sim_stocks_test.to_csv('Stability Analysis - Outputs/Dataframes Optimized/Stock Simulations/df_sim_stocks_test_sim' + str(no_of_paths_test) + '.csv')

#Load the simulated stock dataframes back from csv - to avoid re-running of simualtion
df_sim_stocks = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/Stock Simulations/df_sim_stocks_sim' + str(no_of_paths) + '.csv', index_col=0)
df_sim_stocks_test = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/Stock Simulations/df_sim_stocks_test_sim' + str(no_of_paths_test) + '.csv', index_col=0)


# General Black-Schole Option price of the option to be hedged at all simulated levels of stocks - for both Training and Testing

Taken from 'Stability of Neural Network - Optimised Code'

In [8]:
######################################################
# Generate Blacl-Scholes price for stock simulations #
######################################################

# Black-Scholes function

def d1(S0, r, delta, t0, t, k, sigma):
    return 1/(sigma*np.sqrt(t-t0)) * (np.log(S0/k) + (r-delta+(sigma**2)/2)*(t-t0))

def d2(S0, r, delta, t0, t, k, sigma):
    return 1/(sigma*np.sqrt(t-t0)) * (np.log(S0/k) + (r-delta-(sigma**2)/2)*(t-t0))

def BSMcall(S0, r, delta, t0, t, k, sigma):
    return S0*np.exp(-delta*(t-t0))*norm.cdf(d1(S0, r, delta, t0, t, k, sigma)) - k*np.exp(-r*(t-t0))*norm.cdf(d2(S0, r, delta, t0, t, k, sigma))

def BSMPut(S0, r, delta, t0, t, k, sigma):
    return k*np.exp(-r*(t-t0))*norm.cdf(-d2(S0, r, delta, t0, t, k, sigma)) - S0*np.exp(-delta*(t-t0))*norm.cdf(-d1(S0, r, delta, t0, t, k, sigma))

def fwdValue(S0, r, delta, t0, t, k):
    return S0*np.exp(-delta*(t-t0)) - k*np.exp(-r*(t-t0))

def depositPV(t0, t, p):
    return p*np.exp(-r*(t-t0))
    
#Generate Black-Scholes price for all paths and time grid points - Training Sample
df_bs_price = pd.DataFrame(np.zeros(no_of_paths * (sim_grid_points + 1)).reshape(sim_grid_points + 1, no_of_paths),index=df_sim_stocks.index)
for grid in np.arange(0, sim_grid_points+1):
    df_bs_price.iloc[grid, :] = BSMcall(((df_sim_stocks.iloc[grid,:]).to_numpy()).reshape(1,-1) , r, delta, grid*dt, T, K, sigma)
    
#Generate Black-Scholes price for all paths and time grid points - Testin Sample
df_bs_price_test = pd.DataFrame(np.zeros(no_of_paths * (sim_grid_points + 1)).reshape(sim_grid_points + 1, no_of_paths),index=df_sim_stocks_test.index)
for grid in np.arange(0, sim_grid_points+1):
    df_bs_price_test.iloc[grid, :] = BSMcall(((df_sim_stocks_test.iloc[grid,:]).to_numpy()).reshape(1,-1) , r, delta, grid*dt, T, K, sigma)

# Load the simulated stock dataframes to csv
df_bs_price.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_bs_price_sim' + str(no_of_paths) + '.csv')
df_bs_price_test.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_bs_price_test_sim' + str(no_of_paths_test) + '.csv')

#Load BS Price dataframes from geenrated csv files
# df_bs_price = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_bs_price_sim' + str(no_of_paths) + '.csv', index_col=0)
# df_bs_price_test = pd.read_csv('Stability Analysis - Outputs/Dataframes Optimized/df_bs_price_test_sim' + str(no_of_paths_test) + '.csv', index_col=0)

print("Theoretical PV: ", df_bs_price.iloc[0,0])

Theoretical PV:  0.22461775153220404


# Carr Static Hedge Section

**Generate static hedge portfolio and value the portfolio at time 0 and at simulated levels of stock at termination of short term options**

In [9]:
def Hermitetuple(no_of_options):
    return pd.DataFrame(roots_hermite(no_of_options)).transpose()

def findStrikes(x):
    return K*np.exp(x*sigma*np.sqrt(2*(T-t))+(delta-r-(sigma**2)/2)*(T-t))

def findWeights(x):
    return (np.exp(-delta*(T-t))/np.sqrt(np.pi))*x   

def portfolioStrikes(no_of_options):
    return pd.Series(Hermitetuple(no_of_options)[0].apply(findStrikes))

def portfolioWeights(no_of_options):
    return pd.Series(Hermitetuple(no_of_options)[1].apply(findWeights))

stock_vec = df_sim_stocks_test.iloc[-1, :] 
stock_vec = (stock_vec.to_numpy()).reshape(-1,1)

no_of_options_list = [5, 9, 15, 21]

def find_carr_static_hedge_error(no_of_options_list, stock_vec, df_bs_price_test, no_of_paths_test):
    df_carr_pv_rmse = df_nn_hedge_params = pd.DataFrame(columns=['Options Count','Carr PV', 'Carr RMSE'],  
                                                        index=[i for i in range(0, len(no_of_options_list))])
    df_carr_hedge_error = pd.DataFrame(columns=['Options Count:' + str(no_of_opt) for no_of_opt in no_of_options_list], 
                                       index=[i for i in range(0, no_of_paths_test)])
    
    i = 0
    for no_of_opt in no_of_options_list:
        wts = (portfolioWeights(no_of_options=no_of_opt).to_numpy()).reshape(1,-1)
        strks = (portfolioStrikes(no_of_options=no_of_opt).to_numpy()).reshape(1,-1)
        static_hedge_carr = np.sum(wts*(np.maximum(stock_vec-strks,0)),axis=1)
        static_hedge_carr_error = static_hedge_carr - df_bs_price_test.iloc[-1, :].to_numpy()
        static_hedge_carr_rmse = np.sqrt(np.mean(np.square(static_hedge_carr_error)))
        pvt0_carr = np.sum(wts*BSMcall(S0, r, delta, 0, t, strks, sigma))
        df_carr_pv_rmse.loc[i, ['Options Count','Carr PV', 'Carr RMSE']] = np.array([no_of_opt, pvt0_carr, static_hedge_carr_rmse]).reshape(1,-1)
        i=i+1
        df_carr_hedge_error.loc[:, 'Options Count:' + str(no_of_opt)] = static_hedge_carr_error.reshape(-1,1)
    df_carr_pv_rmse.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_carr_pv_rmse_sim' + str(no_of_paths) + '.csv')
    df_carr_hedge_error.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_carr_hedge_error_sim' + str(no_of_paths) + '.csv')
    
    return None

In [10]:
find_carr_static_hedge_error(no_of_options_list, stock_vec, df_bs_price_test, no_of_paths_test)

In [11]:
no_of_options = 15
print(portfolioStrikes(no_of_options))

0     0.095443
1     0.143332
2     0.202240
3     0.276910
4     0.372577
5     0.495955
6     0.656051
7     0.865291
8     1.141264
9     1.509670
10    2.009591
11    2.703864
12    3.702170
13    5.223728
14    7.844765
Name: 0, dtype: float64


# Original Implementation of Neural Network


In [12]:
#####################################################################
############# Original Implementation of Neural Network #############
#####################################################################

class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, w):        
        return keras_backend.clip(w, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}

def preTrainNeuralNet(df_bs_price, df_sim_stocks, 
                      no_of_options, no_of_epochs, batch_size, 
                      model_type, hidden_bias_init, hidden_weight_init, outer_bias_init):
    
    s_init = df_sim_stocks.iloc[0,0]
        
    nnet_model = Sequential()
    if (model_type == "Constrained Neural Network"):
        ones_array = np.array([1 for i in range(0, no_of_options)]).reshape(1,no_of_options)
        hidden_kernel_constraint = tf.keras.constraints.MinMaxNorm(min_value=1.0, max_value=1.0, rate=1.0, axis=0)
        hidden_layer = Dense(no_of_options, activation='relu', kernel_constraint=hidden_kernel_constraint, 
                             bias_constraint=Between(-999999, 0), input_dim=1)
        hidden_layer.trainable = True
        nnet_model.add(hidden_layer)    
        
        if (hidden_bias_init == "normal0.1"):
            strikes_init = [np.random.normal(1, 0.1) for i in range(0, no_of_options-1)]
            strikes_init.append(1)
            strikes_init = -1 * np.array(strikes_init).reshape(-1)
            
        elif (hidden_bias_init == "normal0.1_seed_fix"):   
            #If Seed is fixed:
            np.random.seed(999)
            strikes_init = list(np.random.normal(1,0.1, size=(no_of_options,)))
            strikes_init = -1 * np.array(strikes_init).reshape(-1)
        
        else:
            strikes_init = np.array(Hermitetuple(no_of_options)[0] + np.amax(Hermitetuple(no_of_options)[0])).reshape(-1)
            strikes_init = strikes_init / np.mean(strikes_init)
        
        hidden_weights = [ones_array, strikes_init]
        nnet_model.layers[0].set_weights(hidden_weights)
        outer_initializer = tf.keras.initializers.Constant(value=(1/no_of_options))
        output_layer = Dense(1, activation='linear', kernel_initializer=outer_initializer, use_bias=False)
        output_layer.trainable = True
        nnet_model.add(output_layer)
    else: 
        if (hidden_bias_init == "opp sign of weight uniform0.5"):
            hidden_layer = Dense(no_of_options, activation='relu', input_dim=1)
            hidden_layer.trainable = True
            nnet_model.add(hidden_layer)    
            strikes_init = np.array(np.random.uniform(low=-0.5, high=0.5, size=no_of_options)).reshape(-1)
            inner_wts = -1 * strikes_init.reshape(1, -1)
            hidden_weights = [inner_wts, strikes_init]
            nnet_model.layers[0].set_weights(hidden_weights)     
        else:
            nnet_model.add(Dense(no_of_options, activation='relu', 
                             kernel_initializer=keras.initializers.RandomUniform(minval=-0.5, maxval=0.5,seed=None)
                             ))
        if (outer_weight_init == "random uniform non-negative"):    
            nnet_model.add(Dense(1, activation='linear', kernel_initializer='random_uniform', kernel_constraint = tf.keras.constraints.NonNeg()))
        else:
            nnet_model.add(Dense(1, activation='linear', kernel_initializer='random_uniform'))
     
    #nnet_model.compile(optimizer=opt.Adam(lr=0.001), loss='mean_squared_error')
    nnet_model.compile(optimizer= tf.optimizers.Adam(lr=0.001), loss='mean_squared_error')
    
    stock_vec = np.array(df_sim_stocks.iloc[-1, :]) / s_init
    bs_value = np.array(df_bs_price.iloc[-1, :]) / s_init
    
    X_train = stock_vec.reshape(-1, 1)
    Y_train = np.asarray(bs_value).reshape(-1, 1)
    
    nnet_output = nnet_model.fit(X_train, Y_train, epochs=no_of_epochs, batch_size=batch_size, verbose=0,
                                 validation_split=0.3)
    return nnet_model
    
def staticHedgeNeural(df_bs_price, df_sim_stocks, pre_nnet_model_orig, 
                                  no_of_options, no_of_epochs, batch_size):
    
    s_init = df_sim_stocks.iloc[0,0]
    stock_vec = np.array(df_sim_stocks.iloc[-1, :]) / s_init
    bs_value = np.array(df_bs_price.iloc[-1, :]) / s_init

    X_train = stock_vec.reshape(-1, 1)
    Y_train = bs_value.reshape(-1, 1)

    nnet_model = pre_nnet_model_orig
    nnet_output = nnet_model.fit(X_train, Y_train, epochs=no_of_epochs, batch_size=batch_size, verbose=0,
                                 validation_split=0.3)
    
    return nnet_model, s_init


def identify_positions(wt_list, strike_list):
    positions = np.where(np.logical_and(wt_list>0, strike_list<0), "C", 
                        np.where(np.logical_and(wt_list>0, strike_list>=0), "F", 
                                 np.where(np.logical_and(wt_list<0, strike_list>0), "P", "N")))
    
    pos = (np.array(np.unique(positions, return_counts=True)).T)
    no_of_pos_dict = {"C":0 , "P":0, "F":0 , "N":0 }
    
    for i in range(0, len(pos[:,0])):
        no_of_pos_dict[pos[i,0]] = int(pos[i,1])

    return no_of_pos_dict["C"], no_of_pos_dict["P"], no_of_pos_dict["F"], no_of_pos_dict["N"], positions

def find_instr_params(nn_strikes, inner_wts, outer_wts, outer_bias, positions, s_init):
    strikes_array = np.where(positions == "F", 
                            - s_init * np.divide(np.abs(nn_strikes), np.abs(inner_wts)), 
                            s_init * np.divide(np.abs(nn_strikes), np.abs(inner_wts)))
    deposit = outer_bias * s_init
    pfl_weights = np.multiply(np.abs(inner_wts), outer_wts)
    
    return strikes_array, deposit, pfl_weights

def static_hedge_value_nn(stock_vec, strike_list, positions, pfl_wts, deposit, 
                          s_init, r, delta, sigma, t):
    strks = strike_list.reshape(1,-1)
    non_neg_strks = np.abs(strks)
    wts = pfl_wts.reshape(1,-1)
    pos = positions.reshape(1,-1)
    c_wt =   np.multiply(wts, np.where(pos=="C", 1, 0))
    p_wt =   np.multiply(wts, np.where(pos=="P", 1, 0))
    f_wt =   np.multiply(wts, np.where(pos=="F", 1, 0))
    
    stock_vec = stock_vec.reshape(-1,1)
    nn_static_hedge_value = np.sum(c_wt*(np.maximum(stock_vec-non_neg_strks,0)) + 
                             p_wt*(np.maximum(non_neg_strks-stock_vec,0)) + 
                             f_wt*(stock_vec-strks), axis=1) + deposit
    
    pvt0_nn = np.sum(c_wt*BSMcall(s_init, r, delta, 0, t, non_neg_strks, sigma) + 
                     p_wt*BSMcall(s_init, r, delta, 0, t, non_neg_strks, sigma) + 
                     f_wt*fwdValue(s_init, r, delta, 0, t, strks), axis=1) + deposit * np.exp(-r*t)
    
    return nn_static_hedge_value, pvt0_nn
    

In [13]:
#Generate Static Hedge using Neural Network

def generate_nn_hedge(df_sim_stocks, df_sim_stocks_test, df_bs_price, df_bs_price_test, 
                      no_of_options, r, delta, sigma, t, 
                     model_type, hidden_bias_init, hidden_weight_init, outer_bias_init):
    batch_size_divisor = 10
    no_of_epochs = 100  
    batch_size = int(no_of_paths/batch_size_divisor)
    pre_nnet_model_orig = preTrainNeuralNet(df_bs_price, df_sim_stocks, no_of_options, no_of_epochs, batch_size, 
                                                    model_type, hidden_bias_init, hidden_weight_init, outer_bias_init)
    no_of_epochs = 1000 
    nnet_model_orig, s_init = staticHedgeNeural(df_bs_price, df_sim_stocks, pre_nnet_model_orig, no_of_options, no_of_epochs, batch_size)
    
    #Neural Network Weights
    inner_wts = np.array(nnet_model_orig.layers[0].get_weights()[0]).reshape(-1)
    nn_strikes_orig = np.array(nnet_model_orig.layers[0].get_weights()[1]).reshape(-1)
    outer_wts = np.array(nnet_model_orig.layers[1].get_weights()[0]).reshape(-1)
    if (outer_bias_init == "no bias"):
        outer_bias = np.array([0]).reshape(-1)
    else:
        outer_bias = np.array(nnet_model_orig.layers[1].get_weights()[1]).reshape(-1)
    
    #Genererate Derivative contract equivalents from weights
    global no_of_calls, no_of_puts, no_of_fwds, nulls, positions, strike_arry, deposit, pfl_wts
    no_of_calls, no_of_puts, no_of_fwds, no_of_nulls, positions = identify_positions(inner_wts, nn_strikes_orig)
    strike_array, deposit, pfl_wts = find_instr_params(nn_strikes_orig, inner_wts, outer_wts, outer_bias, positions, s_init)
    deposit_list = np.array([deposit for i in range(0, no_of_options)])
    params = [strike_array.reshape(no_of_options,1), deposit_list.reshape(no_of_options,1), pfl_wts.reshape(no_of_options,1)]
    
    #Generate Hedge Values
    stock_vec = np.array(df_sim_stocks_test.iloc[-1,:]).reshape(-1)
    nn_static_hedge_value, pvt0_nn = static_hedge_value_nn(stock_vec, strike_array, positions, pfl_wts, deposit, 
                                                           s_init, r, delta, sigma, t)
    theor_bs_price = np.array(df_bs_price_test.iloc[-1,:]).reshape(-1)
    nn_static_hedge_err = nn_static_hedge_value - theor_bs_price
    nn_rmse = np.sqrt(np.mean(np.square(nn_static_hedge_err)))
    
    return pvt0_nn, nn_rmse, nn_static_hedge_err, [no_of_calls, no_of_puts, no_of_fwds, no_of_nulls], positions, params


def load_runs_to_df(df_sim_stocks, df_sim_stocks_test, df_bs_price, df_bs_price_test, 
                    no_of_options, r, delta, sigma, t, 
                    model_type, hidden_bias_init, hidden_weight_init, outer_bias_init,
                    no_of_runs=4):
    df_nn_hedge_pv_instr_nos = pd.DataFrame(columns=['PV', 'NN RMSE', 'Calls Count', 'Puts Count', 'Fwds Count', 'Nulls Count'], 
                                  index=[i for i in range(1, no_of_runs+1)])
    df_nn_hedge_params = pd.DataFrame(columns=['Runs', 'Positions', 'Strike', 'Pfl Wts', 'Deposit'], 
                                  index=[i for i in range(1, no_of_runs*no_of_options + 1)])
    df_nn_hedge_error = pd.DataFrame(columns=['Run' + str(i) for i in range(1, no_of_runs + 1)], 
                                  index=[i for i in range(1, no_of_paths + 1)])

    for run in range(1, no_of_runs+1):
        pvt0_nn, nn_rmse, nn_static_hedge_err, instr_nos, positions, params = generate_nn_hedge(df_sim_stocks, df_sim_stocks_test, df_bs_price, df_bs_price_test, 
                                                                               no_of_options, r, delta, sigma, t,
                                                                               model_type, hidden_bias_init, hidden_weight_init, outer_bias_init)

        positions = positions.reshape(no_of_options,1)
        run_ind_arr = np.array(['Run' + str(run) for i in range(0, no_of_options)]).reshape(no_of_options,1)
        df_nn_hedge_pv_instr_nos.loc[run, ['PV', 'NN RMSE', 'Calls Count', 'Puts Count', 'Fwds Count', 'Nulls Count']] = np.array(list(pvt0_nn) + list(np.array(nn_rmse).reshape(-1)) + instr_nos).reshape(1, 6)
#         df_nn_hedge_params.loc[((run-1)*no_of_options + 1): ((run)*no_of_options), ['Runs', 'Positions', 'Strike', 'Pfl Wts', 'Deposit']] = np.transpose(np.array([run_ind_arr] + [positions] + params).reshape(no_of_options, 5))
        df_nn_hedge_params.loc[((run-1)*no_of_options + 1): ((run)*no_of_options), ['Runs', 'Positions', 'Strike', 'Pfl Wts', 'Deposit']] = np.transpose(np.array([run_ind_arr] + [positions] + params).reshape(5, no_of_options))
        df_nn_hedge_error.loc[:, run_ind_arr[0]] = nn_static_hedge_err.reshape(-1,1)
    
    return df_nn_hedge_pv_instr_nos, df_nn_hedge_params, df_nn_hedge_error


In [14]:
#Can Take following values: 1. "UnConstrained Neural Network", 2. "Constrained Neural Network"
model_type = "UnConstrained Neural Network"

#Type of Bias Initialisation: 1. "normal0.1", 2. "gauss adjusted strikes", 
#3."opp sign of weight uniform0.5", 4."default", 5. "normal0.1_seed_fix"
hidden_bias_init = "default"

#Type of hidden weight initialisation: 1. "uniform0.5", 2. "ones constraint" 
hidden_weight_init = "uniform0.5"

#Outer Weights: 1. "random uniform", 2. "equal weights", 3."high centred", 
#               4."random uniform non-negative"
outer_weight_init = "random uniform"

#Outer Bias: 1. "default", 2."no bias", 3. "non negative"
outer_bias_init = "default"

In [15]:
df_sim_stocks.iloc[-1, :]/df_sim_stocks.iloc[0,0]

0      1.172575
1      1.179842
2      0.844308
3      0.835165
4      1.118043
         ...   
495    1.457625
496    1.346783
497    0.843687
498    1.094644
499    1.007824
Name: 100, Length: 500, dtype: float64

In [16]:
%%time
df_nn_hedge_pv_instr_nos, df_nn_hedge_params, df_nn_hedge_error = load_runs_to_df(df_sim_stocks, df_sim_stocks_test, df_bs_price, df_bs_price_test, 
                                                                                  no_of_options, r, delta, sigma, t,
                                                                                  model_type, hidden_bias_init, hidden_weight_init, outer_bias_init,
                                                                                  no_of_runs=no_of_runs)

C:\Users\JWant\anaconda3\envs\tf2.4\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.

C:\Users\JWant\AppData\Local\Temp\ipykernel_14816\2110650919.py:8: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\JWant\AppData\Local\Temp\ipykernel_14816\2110650919.py:11: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\JWant\anaconda3\envs\tf2.4\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.

C:\Users\JWant\AppData\Local\Temp\ipykernel_14816\2110650919.py:8: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\JWant\AppData\Local\Temp\ipykernel_14816\2110650919.py:11: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\JWant\anaconda3\envs\tf2.4\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.

C

CPU times: total: 2min 48s
Wall time: 2min 12s


C:\Users\JWant\AppData\Local\Temp\ipykernel_14816\2110650919.py:8: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\JWant\AppData\Local\Temp\ipykernel_14816\2110650919.py:11: RuntimeWarning:

divide by zero encountered in true_divide



In [17]:
file_ident = "constr_normal"

In [18]:
df_nn_hedge_pv_instr_nos

,PV,NN RMSE,Calls Count,Puts Count,Fwds Count,Nulls Count
1,0.224408,0.004379,8.0,0.0,0.0,7.0
2,0.224458,0.001528,12.0,0.0,0.0,3.0
3,0.224715,0.003166,6.0,0.0,0.0,9.0
4,0.225477,0.004162,6.0,0.0,0.0,9.0


In [15]:
df_nn_hedge_pv_instr_nos.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_pv_instr_nos_' + file_ident + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv')
df_nn_hedge_params.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_params_' + file_ident + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv')
df_nn_hedge_error.to_csv('Stability Analysis - Outputs/Dataframes Optimized/df_nn_hedge_error_' + file_ident + '_sim' + str(no_of_paths_test) + '_opt' + str(no_of_options) + '_runs' + str(no_of_runs) + '.csv')